In [719]:
from sklearn.datasets import load_wine
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (load_wine())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])

print(x.shape,y.shape)    

# 분류모델이기 때문에 y를 원핫 인코딩변환 해주어야합니다.
y = to_categorical(y)
# print(x.describe)

(178, 13) (178, 1)


In [720]:

print(x.shape,y.shape) # (178, 13) (178, 1)

# y의 클래스 값 분류 + 불균형확인
print(np.unique(y)) # [0 1 2]
print(np.unique(y,return_counts=True)) # [0 1 2]
# 
print(np.unique(y)) # [0 1 2]
print(np.unique(y,return_counts=True)) # [0 1 2]

(178, 13) (178, 3)
[0. 1.]
(array([0., 1.], dtype=float32), array([356, 178], dtype=int64))
[0. 1.]
(array([0., 1.], dtype=float32), array([356, 178], dtype=int64))


In [721]:

# print(x.info())
# print(x.describe())
# 데이터 분리
from sklearn.preprocessing import MinMaxScaler,StandardScaler

x_train, x_test, y_train,y_test = train_test_split(x,y,
                                                   train_size=0.8,
                                                   shuffle = True,
                                                #  stratify 는 데이터 불균형을 해결해줌
                                                   stratify=y
                                                #    random_state=21
                                                   )


# x_train 을 스케일링
# scaler = StandardScaler()
# scaler.fit(x_train)
# x_train_scaled = scaler.transform(x_train)
# # x_test 를 스케일링
# x_test = scaler.transform(x_test)


print(y_train[:5])
print("x_shape : ",x_train.shape)
print("x_shape : ",x_test.shape)
print("y_shape : ",y_train.shape)
print("y_shape : ",y_test.shape)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
x_shape :  (142, 13)
x_shape :  (36, 13)
y_shape :  (142, 3)
y_shape :  (36, 3)


In [722]:
model = Sequential([
    Dense(16,activation="relu",input_shape=(13,)),
    Dense(32,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(32,activation="relu") ,
    Dense(16,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(3,activation="softmax")
])

In [723]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
min_delta=0.01, patience=15, 
verbose=1, 
mode='min')


# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=100, batch_size=8, 
                 validation_split=0.2,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/100
15/15 [==============================] - 0s 13ms/step - loss: 4.6611 - accuracy: 0.3628 - val_loss: 2.6182 - val_accuracy: 0.4138
Epoch 2/100
15/15 [==============================] - 0s 6ms/step - loss: 1.5544 - accuracy: 0.5044 - val_loss: 1.3347 - val_accuracy: 0.4483
Epoch 3/100
15/15 [==============================] - 0s 6ms/step - loss: 1.1086 - accuracy: 0.4779 - val_loss: 0.9226 - val_accuracy: 0.5862
Epoch 4/100
15/15 [==============================] - 0s 6ms/step - loss: 0.8640 - accuracy: 0.6903 - val_loss: 0.7731 - val_accuracy: 0.6897
Epoch 5/100
15/15 [==============================] - 0s 5ms/step - loss: 1.1224 - accuracy: 0.6018 - val_loss: 0.7170 - val_accuracy: 0.7931
Epoch 6/100
15/15 [==============================] - 0s 5ms/step - loss: 1.0109 - accuracy: 0.5929 - val_loss: 0.8977 - val_accuracy: 0.5172
Epoch 7/100
15/15 [==============================] - 0s 5ms/step - loss: 0.8462 - accuracy: 0.5664 - val_loss: 0.7044 - val_accuracy: 0.8276
Epoch 8/100


In [724]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


2/2 [==============================] - 0s 3ms/step - loss: 0.4282 - accuracy: 0.8333
loss: 0.42824646830558777 
acc : 0.8333333134651184


In [725]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)

# predict 의 값은 총 3개가 나오는데 softmax 특성상 sum 의 값은 1이 됨
print(y_predict[:5])

# argmax 는 x 를 최대값으로 만들어주는 입력을 구하는함수
y_predict = np.argmax(y_predict,axis=1)
# y_predict = to_categorical(y_predict,3)
print("예측 : ", y_predict[:15])
# y_test 를 원핫 인코딩 해제 해야함
y_test = np.argmax(y_test,axis=1)
print("실제 : ", y_test[:15])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print("acc : ",acc)

[[9.1223955e-01 8.7648980e-02 1.1150778e-04]
 [1.5470512e-03 4.6673841e-03 9.9378562e-01]
 [9.8328453e-01 1.6715214e-02 2.1373363e-07]
 [8.3323389e-01 1.6669565e-01 7.0423281e-05]
 [1.7923813e-01 1.5614006e-01 6.6462183e-01]]
예측 :  [0 2 0 0 2 1 1 0 2 2 1 1 0 0 1]
실제 :  [0 2 0 0 2 1 1 0 2 2 0 1 0 0 1]
acc :  0.8333333333333334
